In [29]:
import os
import pandas as pd
import naiveautoml
import numpy as np

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../.."))
DATA_PATH = os.path.join(BASE_DIR, "dataset", "data", "premier_dataset_final.csv")

df = pd.read_csv(DATA_PATH)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").reset_index(drop=True)
split_idx = len(df) // 2

train_df = df.iloc[:split_idx]
valid_df = df.iloc[split_idx:]

X_train = train_df.drop(columns=["result", "date"])
y_train = train_df["result"]

X_valid = valid_df.drop(columns=["result", "date"])
y_valid = valid_df["result"]

categorical_features = [c for c in ["season", "home_team", "away_team", "is_first_half", "season_half"]
                        if c in X_train.columns]
categorical_features

['season', 'home_team', 'away_team', 'is_first_half', 'season_half']

In [40]:
naml = naiveautoml.NaiveAutoML(
    show_progress=True,
                    
)

naml.fit(X_train, y_train, categorical_features=categorical_features, )

Progress for algorithm selection:


0.02s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.02s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.01s - Debugger warning: It seems that frozen modules are being used, which may
0

Progress for hyperparameter optimization:


100%|██████████| 100/100 [00:22<00:00,  4.44it/s]


In [47]:
from sklearn.metrics import accuracy_score, f1_score, log_loss, classification_report, confusion_matrix

model = naml.chosen_model
y_pred = model.predict(X_valid)
y_proba = model.predict_proba(X_valid) if hasattr ( model, "predict_proba") else None

acc = accuracy_score(y_valid, y_pred)
f1m = f1_score(y_valid, y_pred, average="macro")
print(f"\nAccuracy: {acc:.3f}")
print(f"F1-macro: {f1m:.3f}")


if y_proba is not None:
    try:
        ll = log_loss(y_valid, y_proba, labels=sorted(y_valid.unique()))
        print(f" LogLoss: {ll:.3f}")
    except Exception:
        pass

print("\nReporte de clasificación:")
print(classification_report(y_valid, y_pred))

print(" Matriz de confusión:")
print(confusion_matrix(y_valid, y_pred))


Accuracy: 0.916
F1-macro: 0.898
 LogLoss: 3.035

Reporte de clasificación:
              precision    recall  f1-score   support

           A       0.94      0.96      0.95        70
           D       0.82      0.78      0.80        41
           H       0.94      0.95      0.94        79

    accuracy                           0.92       190
   macro avg       0.90      0.90      0.90       190
weighted avg       0.91      0.92      0.92       190

 Matriz de confusión:
[[67  3  0]
 [ 4 32  5]
 [ 0  4 75]]
